In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.8 MB/s eta 0:00:00a 0:00:01


In [5]:
import os, cv2, shutil, yaml
import numpy as np
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
BASE_DIR = "/kaggle/input/nutrition5k-dataset/imagery/realsense_overhead"
DATA_DIR = "/kaggle/working/data"
img_dir, lbl_dir = f"{DATA_DIR}/images", f"{DATA_DIR}/labels"

for p in [img_dir, lbl_dir]: os.makedirs(p, exist_ok=True)

In [7]:
def generate_smart_label(depth_path):
    d_img = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)
    if d_img is None: return None
    
    blurred = cv2.GaussianBlur(d_img, (5, 5), 0)
    _, mask = cv2.threshold(blurred.astype(np.uint8), 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    yolo_data = []
    h, w = mask.shape
    for c in cnts:
        if cv2.contourArea(c) < 500: continue
        pts = c.flatten().tolist()
        norm = [str(p/w if i%2==0 else p/h) for i, p in enumerate(pts)]
        yolo_data.append(f"0 " + " ".join(norm))
    return yolo_data

In [8]:
dish_folders = sorted(os.listdir(BASE_DIR))[:2500]
count = 0

for dish_id in dish_folders:
    d_path = os.path.join(BASE_DIR, dish_id)
    rgb, depth = os.path.join(d_path, "rgb.png"), os.path.join(d_path, "depth_raw.png")
    
    if os.path.exists(rgb) and os.path.exists(depth):
        labels = generate_smart_label(depth)
        if labels:
            if not os.path.exists(f"{img_dir}/{dish_id}.png"):
                os.symlink(rgb, f"{img_dir}/{dish_id}.png")
            with open(f"{lbl_dir}/{dish_id}.txt", "w") as f:
                f.write("\n".join(labels))
            count += 1

print(f"{count} images were ready with 'Smart Labeling'")

2480 images were ready with 'Smart Labeling'


In [9]:
config = {'path': DATA_DIR, 'train': 'images', 'val': 'images', 'nc': 1, 'names': ['food']}
with open('/kaggle/working/food.yaml', 'w') as f: yaml.dump(config, f)

In [10]:
model = YOLO('yolo11n-seg.pt')

model.train(
    data='/kaggle/working/food.yaml',
    epochs=100,
    imgsz=640,
    batch=64,
    device=[0, 1],
    workers=8,
    mosaic=1.0,
    mixup=0.1,
    copy_paste=0.3,       
    project='nutrient_ai_pro',
    name='final_model',
    exist_ok=True,
    amp=True
)

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/food.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=final_model, nbs=64, nms=False, opset=None, optimize=Fa

In [12]:
model_path = '/kaggle/working/nutrient_ai_pro/final_model/weights/best.pt'

model = YOLO(model_path)

model.export(format='coreml', nms=True, int8=True)

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
WARNING ⚠️ INT8 export requires a missing 'data' arg for calibration. Using default 'data=coco8-seg.yaml'.
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11n-seg summary (fused): 113 layers, 2,834,763 parameters, 0 gradients, 9.6 GFLOPs

PyTorch: starting from '/kaggle/working/nutrient_ai_pro/final_model/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 37, 8400), (1, 32, 160, 160)) (5.7 MB)
requirements: Ultralytics requirement ['coremltools>=9.0'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 12 packages in 274ms
Prepared 2 packages in 163ms
Installed 2 packages in 18ms
 + cattrs==25.3.0
 + coremltools==9.0

requirements: AutoUpdate success ✅ 0.9s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


scikit-learn version 1.6.1 is not supported. Minimum required version: 0.17. Maximum required version: 1.5.1. Disabling scikit-learn conversion API.
XGBoost version 3.1.0 has not been tested with coremltools. You may run into unexpected errors. XGBoost 1.4.2 is the most recent version that has been tested.
E0000 00:00:1766867326.136457      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766867326.188282      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766867326.644424      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766867326.644448      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more 


CoreML: starting export with coremltools 9.0...
WARNING ⚠️ CoreML: 'nms=True' is only available for Detect models like 'yolo11n.pt'.


Running MIL default pipeline:   9%|▉         | 9/95 [00:00<00:00, 86.14 passes/s]/usr/local/lib/python3.12/dist-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:273: UserWarning: Output, '1365', of the source model, has been renamed to 'var_1365' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running compression pass palettize_weights:  97%|█████████▋| 194/199 [01:03<00:00,  5.79 ops/s]/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (256). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
Running compression pass palettize_weights: 100%|██████████| 199/199 [01:04<00:00,  3.10 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 65.22 passes/s]


CoreML: export success ✅ 92.6s, saved as '/kaggle/working/nutrient_ai_pro/final_model/weights/best.mlpackage' (3.0 MB)

Export complete (93.2s)
Results saved to /kaggle/working/nutrient_ai_pro/final_model/weights
Predict:         yolo predict task=segment model=/kaggle/working/nutrient_ai_pro/final_model/weights/best.mlpackage imgsz=640 int8 
Validate:        yolo val task=segment model=/kaggle/working/nutrient_ai_pro/final_model/weights/best.mlpackage imgsz=640 data=/kaggle/working/food.yaml int8 
Visualize:       https://netron.app


'/kaggle/working/nutrient_ai_pro/final_model/weights/best.mlpackage'

In [13]:
path_to_model = '/kaggle/working/nutrient_ai_pro/final_model/weights/best.mlpackage'
output_filename = '/kaggle/working/nutrient_ai_pro_M4'

shutil.make_archive(output_filename, 'zip', path_to_model)
print(f"{output_filename}.zip")

/kaggle/working/nutrient_ai_pro_M4.zip


In [14]:
results_path = '/kaggle/working/nutrient_ai_pro/final_model'
shutil.make_archive('model_evaluation_stats', 'zip', results_path)

'/kaggle/working/model_evaluation_stats.zip'